In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import ElasticNet
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel
from sklearn import linear_model
import sklearn.feature_selection as fs
import sklearn.linear_model as lin_model
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from statistics import mean

In this exercise, we use the leukemia data X_gol with 3562 features and the breast cancer data set X_b with 27 features.

In [2]:
X_gol = pd.read_csv("data/Golub_X",sep=" ",header=None) # Observations
y_gol = pd.read_csv("data/Golub_y",sep=" ",header=None) # Classes
y_gol=np.array(y_gol).reshape(-1)
X_gol.shape,y_gol.shape

((72, 3562), (72,))

In [24]:
X_b = pd.read_csv("data/Breast.txt",sep=",",header=None)
y_b = X_b.values[:,1] # Classes
X_b = X_b.values[:,2:29] # Observations

y_b[y_b=="M"]=0
y_b[y_b=="B"]=1
y_b=y_b.astype(np.int64)

X_b.shape,y_b.shape



((569, 27), (569,))

# Feature selection - Heuristic Approach

Here, we delete all features whose variance is less than a given threshold. In this ways, we focus on those features which are sigficantly different through the observations. Then, we compare the accuracy of a Logisitic Regression with data consisting of all features and only the selected features.

In [5]:
X=X_gol
y=y_gol 

#X=X_b
#y=y_b

threshold=0.068
#threshold=20

selector=fs.VarianceThreshold(threshold)
X_thres=selector.fit_transform(X)

print("Number of selected features: %d from %d" % (X_thres.shape[1],X.shape[1]) ) 

model_sel=lin_model.LogisticRegressionCV(penalty="l2",cv=3,max_iter=10000)
model_sel.fit(X_thres,y)
predictions_sel=model_sel.predict(X_thres)

model=lin_model.LogisticRegressionCV(penalty="l2",cv=3,max_iter=10000)
model.fit(X,y)
predictions=model.predict(X)


print(  "Accuracy on selected data set %.3f\nAccuracy on original data set %.3f" % 
        (accuracy_score(y,predictions_sel),accuracy_score(y,predictions)))

Number of features left: 7 from 3562
Accuracy on selected data set 0.694
Accuracy on original data set 1.000


# Feature Selection - Statistical Tests

Now, we delete all features which are not statistically significant. So all features with a p-value greater than the threshold alpha will be deleted.

In [30]:
X=X_gol
y=y_gol 

X=X_b
y=y_b

alpha=10e-6
alpha=10e-8
selector=fs.SelectFdr(alpha=alpha)
X_thres=selector.fit_transform(X,y)

#print(selector.pvalues_)

print("Number of selected features: %d from %d" % (X_thres.shape[1],X.shape[1]) ) 


model_sel=lin_model.LogisticRegressionCV(penalty="l2",cv=5,max_iter=10000)
model_sel.fit(X_thres,y)
predictions_sel=model_sel.predict(X_thres)

model=lin_model.LogisticRegressionCV(penalty="l2",cv=5,max_iter=10000)
model.fit(X,y)
predictions=model.predict(X)


print(  "Accuracy on selected data set %.3f\nAccuracy on original data set %.3f" % 
        (accuracy_score(y,predictions_sel),accuracy_score(y,predictions)))

Number of features left: 22 from 27
Accuracy on selected data set 0.982
Accuracy on original data set 0.988


# Feature Selection within the model

We can also choose sparse models which automatically zero out some features. Here, we use the Lasso model, Elastic net model and a support vector machine penalized by the L1 term. <br><br>
Since the leukemia data set has relatively few observations (72), the model performance on test set depends quite heavily on the choice of the test set. That is why we choose 10 times a train and test model and take the mean accuracy.


In [35]:
acc_Lasso=[]
sel_feat_Lasso=[]
acc_SVM=[]
sel_feat_SVM=[]
acc_ela=[]
sel_feat_ela=[]

X=X_gol
y=y_gol 

#X=X_b
#y=y_b

alpha=0.03
C=0.1

for _ in range(10):
        X_tr,X_te,y_tr,y_te=train_test_split(X,y,test_size=0.2,shuffle=True)

        Lasso=lin_model.Lasso(alpha=alpha,max_iter=10000)
        Lasso.fit(X_tr,y_tr)
        predictions_Lasso=Lasso.predict(X_te)
        acc_Lasso.append(accuracy_score(y_te,predictions_Lasso.astype(np.int64)))
        sel_feat_Lasso.append(np.where(Lasso.coef_!=0)[0].shape[0])
        
        SVM=LinearSVC(C=C, penalty="l1", dual=False,max_iter=10000)
        SVM.fit(X_tr,y_tr)
        predictions_SVM=SVM.predict(X_te)
        acc_SVM.append(accuracy_score(y_te,predictions_SVM))
        sel_feat_SVM.append(np.where(SVM.coef_!=0)[0].shape[0])

        Ela_net=lin_model.ElasticNet(alpha=alpha,l1_ratio=0.7,max_iter=10000)
        Ela_net.fit(X_tr,y_tr)
        predictions_ela=Ela_net.predict(X_te)
        acc_ela.append(accuracy_score(y_te,predictions_ela.astype(np.int64)))
        sel_feat_ela.append(np.where(Ela_net.coef_!=0)[0].shape[0])

print("---Lasso---")
print(  "Accuracy: %.3f\t Number of selected features %d" % (mean(acc_Lasso),mean(sel_feat_Lasso)))
print("---SVM---")
print(  "Accuracy: %.3f\t Number of selected features %d" % (mean(acc_SVM),mean(sel_feat_SVM)))
print("---Elastic Net---")
print(  "Accuracy: %.3f\t Number of selected features %d" % (mean(acc_ela),mean(sel_feat_ela)))

---Lasso---
Accuracy: 0.653	 Number of selected features 9
---SVM---
Accuracy: 0.853	 Number of selected features 8
---Elastic Net---
Accuracy: 0.660	 Number of selected features 18


We see that Lasso is obviously sparser than Elastic net since we use a L1-ratio of 0.7.<br>
However, we see that their performance is very similar on both data sets. The best model among them is the support vector machine, also on both dat sets.